SERVER CLASS

In [11]:
class Server:
    def __init__(self, server_id, ip_address, available_cpu,available_memory,available_bandwidth , network_distance,hosted_services=None):
        self.server_id = server_id
        self.network_distance = network_distance
        self.ip_address = ip_address
        self.available_cpu = available_cpu
        self.available_memory = available_memory
        self.available_bandwidth = available_bandwidth
        self.hosted_services = hosted_services if hosted_services is not None else []

        self.agg_resources = self.calculate_agg_resources()

    def calculate_agg_resources(self):
        """Calculate the average resource utilization as an aggregate metric."""
        return (self.available_cpu + self.available_memory + self.available_bandwidth) / 3

    
    def add_service_function(self, service_function):
        self.hosted_services.append(service_function)

    def remove_service_function(self, service_function):
        self.hosted_services.remove(service_function)

    def list_service_functions(self):
        return self.hosted_services
    
    def calculate_latency_priority(self, microservice):
        """Simulate latency based on network distance and server load, in real life we would have the microservice ping the server."""
        base_latency = self.network_distance * 0.5  # Simplified calculation
        return base_latency
    
    def meets_requirements(self, cpu_requirement, memory_requirement, bandwidth_requirement):
        """Check if the server has enough resources to host the microservice."""
        return (self.available_cpu >= cpu_requirement and
                self.available_memory >= memory_requirement and
                self.available_bandwidth >= bandwidth_requirement)
    def deploy_microservice(self, microservice):
        #Deploy service and subtract required resources
        if not self.meets_requirements(microservice.cpu_requirement, microservice.memory_requirement, microservice.bandwidth_requirement):
            return False  # Deployment fails due to insufficient resources
        
        self.available_cpu -= microservice.cpu_requirement
        self.available_memory -= microservice.memory_requirement
        self.available_bandwidth -= microservice.bandwidth_requirement
        self.hosted_services.append(microservice.service_id)
        microservice.server=self
        return True

    def remove_microservice(self, microservice):
        #remove a service and re-add used resources to server
        if microservice.service_id in self.hosted_services:
            self.available_cpu += microservice.cpu_requirement
            self.available_memory += microservice.memory_requirement
            self.available_bandwidth += microservice.bandwidth_requirement
            self.hosted_services.remove(microservice.service_id)
            microservice.server= None

    def __str__(self):
        return (f"Server(ID={self.server_id}, IP={self.ip_address}, "
                f"CPU={self.available_cpu}, Memory={self.available_memory}, "
                f"Bandwidth={self.available_bandwidth}, Distance={self.network_distance}, "
                f"Hosted_Services={len(self.hosted_services)})")

In [15]:

       
#MICROSERVICE SAMPLE CLASS   
class Microservice:
    def __init__(self, service_id, name, cpu_requirement, memory_requirement, bandwidth_requirement, latency_threshold, server=None):
        self.service_id = service_id
        self.name = name
        self.cpu_requirement = cpu_requirement
        self.memory_requirement = memory_requirement
        self.bandwidth_requirement = bandwidth_requirement
        self.latency_threshold = latency_threshold
        self.server = server  # HOST SERVER

    def __str__(self):
        server_id = self.server.server_id if self.server else "Not deployed"
        return (f"Microservice(name={self.name}, CPU={self.cpu_requirement}, "
                f"Memory={self.memory_requirement}, Bandwidth={self.bandwidth_requirement}, "
                f"Latency Threshold={self.latency_threshold}ms, Server={server_id})")



In [16]:

#HASH-QUEUE  
from queue import PriorityQueue
  
    

class EfficientServerModel:
    def __init__(self):
        self.servers = {}  # Server ID to Server object

    def add_server(self, server):
        self.servers[server.server_id] = server

    def rank_servers_for_microservice(self, microservice):
        """
        Returns a sorted list of servers (from most to least suitable according to latency,
        so servers with unsuitable resources wouldn't be included)

        """
        suitable_servers = []
        for server in self.servers.values():
            if server.meets_requirements(
                microservice.cpu_requirement, 
                microservice.memory_requirement, 
                microservice.bandwidth_requirement
            ):
                priority = server.calculate_latency_priority(microservice)
                suitable_servers.append((priority, server))
        
        # Sort servers by priority(lower latency).
        suitable_servers.sort(key=lambda x: x[0])
        
        # Return only the server objects in sorted order.
        return [server for _, server in suitable_servers]

    def find_optimal_server(self, microservice):
        """
        Find the optimal server for the given microservice.
        """
        ranked_servers = self.rank_servers_for_microservice(microservice)
        if ranked_servers:
            return ranked_servers[0]  # Return the server object with the highest priority
        else:
            return None



In [17]:

#ENVIRONMENT

class MicroserviceDeploymentEnv:
    def __init__(self, servers, microservices):
        self.servers = servers  # List of Server objects
        self.microservices = microservices  # List of Microservice objects
        self.current_microservice = None  # Track the microservice being deployed
        self.state_size = None  # To be defined based on normalization
        self.action_size = len(servers) # Our action will be in the form of an integer(deploy to server N)
    @staticmethod
    def  normalize(value, min_value, max_value):
        """Normalize a value to 0-1 range for optimal dqn input"""

        return (value - min_value) / (max_value - min_value) if max_value > min_value else 0
    
    def calculate_latency_bounds(self, microservice):
        latency_estimates = [
            server.calculate_latency_priority(microservice) for server in self.servers
        ]
        min_latency = min(latency_estimates)
        max_latency = max(latency_estimates)
        return min_latency, max_latency

    def get_state(self):
        min_cpu, max_cpu = float('inf'), -float('inf')
        min_memory, max_memory = float('inf'), -float('inf')
        min_bandwidth, max_bandwidth = float('inf'), -float('inf')
        min_distance, max_distance = float('inf'), -float('inf')

        # Iterate through servers once... update min and max values for each parameter
        for server in self.servers:
            min_cpu, max_cpu = min(min_cpu, server.available_cpu), max(max_cpu, server.available_cpu)
            min_memory, max_memory = min(min_memory, server.available_memory), max(max_memory, server.available_memory)
            min_bandwidth, max_bandwidth = min(min_bandwidth, server.available_bandwidth), max(max_bandwidth, server.available_bandwidth)
            min_distance, max_distance = min(min_distance, server.network_distance), max(max_distance, server.network_distance)

        

        server_states = []
        for server in self.servers:
            server_state = [
                self.normalize(server.available_cpu, min_cpu, max_cpu),
                self.normalize(server.available_memory, min_memory, max_memory),
                self.normalize(server.available_bandwidth, min_bandwidth, max_bandwidth),
                self.normalize(server.network_distance, min_distance, max_distance),
            ]
            server_states.extend(server_state)

        
        min_latency, max_latency = self.calculate_latency_bounds(self.current_microservice)

        microservice_state = [
            self.normalize(self.current_microservice.cpu_requirement, min_cpu, max_cpu),
            self.normalize(self.current_microservice.memory_requirement, min_memory, max_memory),
            self.normalize(self.current_microservice.bandwidth_requirement, min_bandwidth, max_bandwidth),
            self.normalize(self.current_microservice.latency_threshold, min_latency, max_latency),
        ]

   
        return server_states + microservice_state
    

    def calculate_reward(self, server, microservice):
        """Calculate the reward for deploying to a server."""
        
        #Higher reward for lower latency
        latency_reward = 1 / (1 + server.calculate_latency_priority(microservice))

        #Reward based on even distribution of resources across servers
        load_balance_reward = self.evaluate_load_balance()

        #Total Reward
        reward = latency_reward + load_balance_reward
        return reward

    def evaluate_load_balance(self):
        """Evaluate the load balance across all servers."""
        #Load balance metric: Standard deviation of server utilization rates
        utilizations = [server.calculate_agg_resources() for server in self.servers.values()]
        mean_utilization = sum(utilizations) / len(utilizations)
        variance = sum((x - mean_utilization) ** 2 for x in utilizations) / len(utilizations)
        std_deviation = variance ** 0.5

        #Higher reward for lower standard deviation
        return 1 / (1 + std_deviation)
    

    def deploy_microservice_to_server(self, microservice_id, server_id):
        """Deploy a microservice to a specified server and return the reward."""
        microservice = next((ms for ms in self.microservices if ms.service_id == microservice_id), None)
        server = self.servers.get(server_id, None)

        if not microservice or not server:
            return -1  # Negative reward for invalid action

        success = server.deploy_microservice(microservice)
        if success:
            reward = self.calculate_reward(server, microservice)
        else:
            reward = -1  # Negative reward for failed deployment due to lack of resources

        return reward



In [22]:
import random

def sample_test():
    # Initialize servers as a dictionary
    servers = {
        "server1": Server("server1", "192.168.1.1", 100, 256, 1000, 10),
        "server2": Server("server2", "192.168.1.2", 200, 512, 2000, 20),
    }

    # Generating 50 microservices
    microservices = [
        Microservice(f"service{i}", f"Service{i}", random.randint(10, 50), random.randint(64, 128), random.randint(200, 500), random.randint(1, 5))
        for i in range(1, 51)
    ]

    # Create the environment
    env = MicroserviceDeploymentEnv(servers, microservices)
    
    print("Before deployment:")
    for server_id, server in env.servers.items():
        print(server)

    # Deploy each microservice to a randomly chosen server
    for microservice in microservices:
        selected_server_id = random.choice(list(env.servers.keys()))  # Randomly select a server
        reward = env.deploy_microservice_to_server(microservice.service_id, selected_server_id)
        print(f"Deployed {microservice.service_id} to {selected_server_id} with reward: {reward}")

    print("\nAfter deployment:")
    for server_id, server in env.servers.items():
        print(server)

#test
sample_test()


Before deployment:
Server(ID=server1, IP=192.168.1.1, CPU=100, Memory=256, Bandwidth=1000, Distance=10, Hosted_Services=0)
Server(ID=server2, IP=192.168.1.2, CPU=200, Memory=512, Bandwidth=2000, Distance=20, Hosted_Services=0)
Deployed service1 to server1 with reward: 0.16986666666666667
Deployed service2 to server2 with reward: 0.09504987765857331
Deployed service3 to server1 with reward: 0.16979329511898558
Deployed service4 to server1 with reward: -1
Deployed service5 to server2 with reward: 0.09485645933014354
Deployed service6 to server2 with reward: 0.09658016841381681
Deployed service7 to server1 with reward: -1
Deployed service8 to server1 with reward: -1
Deployed service9 to server1 with reward: -1
Deployed service10 to server1 with reward: -1
Deployed service11 to server2 with reward: 0.09927729174591099
Deployed service12 to server2 with reward: 0.15031503150315031
Deployed service13 to server2 with reward: -1
Deployed service14 to server1 with reward: -1
Deployed service15 